In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import networkx as nx

class TrueSkillGraph:
    def __init__(self, graph, beta=1.0):
        self.graph = graph
        self.n_players = len(graph.nodes)
        self.beta = beta
        self.skills = np.zeros(self.n_players)

    def _log_posterior(self, skills, outcomes):
        L = nx.laplacian_matrix(self.graph).toarray()
        smoothness = -0.5 * skills @ L @ skills
        
        # Compute outcome likelihood
        i, j, y = outcomes[:, 0], outcomes[:, 1], outcomes[:, 2]
        skill_diff = skills[i] - skills[j]
        prob = norm.cdf(skill_diff / (np.sqrt(2) * self.beta))
        prob = np.clip(prob, 1e-10, 1 - 1e-10)
        
        log_likelihood = y * np.log(prob) + (1 - y) * np.log(1 - prob)
        return -(smoothness + np.sum(log_likelihood))
    
    def fit(self, outcomes):
        result = minimize(
            self._log_posterior,
            self.skills,
            args=(outcomes,),
            method='L-BFGS-B'
        )
        if result.success:
            self.skills = result.x
        else:
            raise RuntimeError("Optimization failed: " + result.message)

    def predict_prob(self, i, j):
        skill_diff = self.skills[i] - self.skills[j]
        return norm.cdf(skill_diff / (np.sqrt(2) * self.beta))

# Example:
G = nx.Graph()
G.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3), (3, 4)])

outcomes = np.array([
    [0, 1, 1], # player 0 beats player 1
    [2, 3, 0], # player 3 beats player 2
    [1, 3, 1], # player 1 beats player 3
    [0, 4, 0]  # player 4 beats player 0
])

model = TrueSkillGraph(G)
model.fit(outcomes)

# Predict win probability
model.predict_prob(0, 1)

np.float64(0.48886397328433284)